In [2]:
import gurobipy as gp
from gurobipy import GRB
import pandapower as pp
import pandapower.networks as pn

import itertools
from Networks import ieee123

net = ieee123.ieee123()
#net = pn.create_cigre_network_lv()

IEEE network is loaded


c:\Users\tmdck\Dropbox\5. Codes\7. optimizations\2. Reconfiguration\Networks\ieee123.py:102: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  net.sgen.type[net.sgen.type == 'WT'] = 'WP'


In [74]:
for line in net.line.index:
    bus = net.line.loc[line].from_bus
    pp.create_switch(net, bus, line, et = "l", closed = True) # 모든 line에 switch 연결
   

In [75]:
pp.create_switch(net, 4,10, et= 'b', closed = True)

40

In [3]:
Buses = list(net.bus.index)
SubstationBuses = list(net.ext_grid.bus)
NonsubstationBuses = list(net.bus.index)

for bus in SubstationBuses:
    NonsubstationBuses.remove(bus)

G, B = dict(), dict()
for bus in Buses:
    G[bus] = 0
    B[bus] = 0

Lines = []
R, X = dict(), dict()
for idx in net.line.index:
    col = net.line.loc[idx]
    Lines.append((col.from_bus, col.to_bus))
    Lines.append((col.to_bus, col.from_bus))
    
    R[(col.from_bus, col.to_bus)] = col.r_ohm_per_km*col.length_km
    R[(col.to_bus, col.from_bus)] = col.r_ohm_per_km*col.length_km
    
    X[(col.from_bus, col.to_bus)] = col.x_ohm_per_km*col.length_km
    X[(col.to_bus, col.from_bus)] = col.x_ohm_per_km*col.length_km
    
# switch 만 연결된 line 도 line임

for idx in net.switch.index:
    col = net.switch.loc[idx]
    if col.et == 'b':
        Lines.append((col.bus, col.element))
        Lines.append((col.element, col.bus))
        R[(col.bus, col.element)] = 0.00001
        R[(col.element, col.bus)] = 0.00001
        
        X[(col.bus, col.element)] = 0.00001
        X[(col.element, col.bus)] = 0.00001

    if col.et == 't':
        from_bus = net.trafo.loc[net.switch.loc[idx].element].hv_bus
        to_bus = net.trafo.loc[net.switch.loc[idx].element].lv_bus
        
        if (from_bus, to_bus) not in Lines:
            Lines.append((from_bus, to_bus))
        if (to_bus, from_bus) not in Lines:
            Lines.append((to_bus, from_bus))
        
        R[(to_bus, from_bus)] = 0.00001
        R[(from_bus, to_bus)] = 0.00001

        X[(col.bus, col.element)] = 0.00001
        X[(col.element, col.bus)] = 0.00001
        
sw = dict()
for idx in net.switch.index:
    
    from_bus = net.switch.loc[idx].bus
    to_bus = None
    if net.switch.loc[idx].et == 'l':
        if net.line.loc[net.switch.loc[idx].element].to_bus != from_bus:
            to_bus = net.line.loc[net.switch.loc[idx].element].to_bus
        else:
            to_bus = net.line.loc[net.switch.loc[idx].element].from_bus
    if net.switch.loc[idx].et == 'b':
        to_bus = net.switch.loc[idx].element
    
    if net.switch.loc[idx].et == 't':
        to_bus = net.trafo.loc[net.switch.loc[idx].element].lv_bus
        
    sw[idx] = (from_bus, to_bus)
    
Switches = []
for switch in sw.values():
    if switch not in Switches:
        Switches.append(switch)
    if tuple(reversed(switch)) not in Switches:
        Switches.append(tuple(reversed(switch)))

from copy import deepcopy

Switches = []
for switch in sw.values():
    if switch not in Switches:
        Switches.append(switch)
    if tuple(reversed(switch)) not in Switches:
        Switches.append(tuple(reversed(switch)))

NoSwitches = deepcopy(Lines)

for switch in Switches:
    NoSwitches.remove(switch)


def build_adjacency_list(edges):
    adjacency_list = {}
    for node1, node2 in edges:
        if node1 not in adjacency_list:
            adjacency_list[node1] = []
        if node2 not in adjacency_list:
            adjacency_list[node2] = [] 
        #adjacency_list[node1].append(node2)
        adjacency_list[node2].append(node1)  # If the graph is undirected

    return adjacency_list
AdjacencyList = build_adjacency_list(Lines)


P = dict()
Q = dict()
for bus in Buses:
    P[bus] = 0
    Q[bus] = 0
    
for idx in net.load.index:
    bus = net.load.bus.loc[idx]
    p = net.load.p_mw.loc[idx]*net.load.scaling.loc[idx]
    q = net.load.q_mvar.loc[idx]*net.load.scaling.loc[idx]
    
    P[bus] += p
    Q[bus] += q
    
for idx in net.sgen.index:
    bus = net.sgen.bus.loc[idx]
    p = net.sgen.p_mw.loc[idx]*net.sgen.scaling.loc[idx]
    q = net.sgen.q_mvar.loc[idx]*net.sgen.scaling.loc[idx]
    
    P[bus] -= p
    Q[bus] -= q

In [11]:
AdjacencyList

{0: [1, 2, 6, 115],
 1: [0],
 2: [0, 3, 4],
 6: [0, 7],
 3: [2],
 4: [2, 5],
 5: [4],
 7: [6, 11, 8, 12],
 11: [7],
 8: [7, 13],
 12: [7, 33, 17, 117],
 13: [8, 10, 9],
 33: [12, 14],
 17: [12, 18, 20, 114],
 10: [13],
 9: [13],
 14: [15, 16, 33],
 15: [14],
 16: [14, 95],
 18: [17, 19],
 20: [17, 21, 22],
 19: [18],
 21: [20],
 22: [20, 23, 24],
 23: [22],
 24: [22, 25, 27],
 25: [24, 26, 30],
 27: [24, 28],
 26: [25, 32],
 30: [25, 31],
 32: [26, 124],
 28: [27, 29],
 29: [28, 120],
 120: [29],
 31: [30],
 34: [35, 39, 114],
 35: [34, 36, 37],
 39: [34, 40, 41],
 36: [35],
 37: [35, 38],
 38: [37, 65],
 40: [39],
 41: [39, 42, 43],
 42: [41],
 43: [41, 44, 46],
 44: [43, 45],
 46: [43, 47, 48],
 45: [44],
 47: [46],
 48: [46, 49],
 49: [48, 50],
 50: [49, 116],
 116: [50, 121],
 51: [52, 117],
 52: [51, 53],
 53: [52, 54, 56, 93],
 54: [53, 55],
 56: [53, 57, 59],
 55: [54],
 57: [56, 58],
 59: [56, 60, 61, 118],
 58: [57],
 60: [59, 125],
 61: [59, 62],
 62: [61, 63],
 63: [62, 64],

In [46]:
    # Define the sets of generators and buses (nodes)
Buses = [1,2,3]
NonsubstationBuses = [2,3]
SubstationBuses = [1]
Lines = [(1,2),(1,3),(2,1),(3,1), (2,3),(3,2)] 
Switches = [(1,2),(1,3),(2,1),(3,1), (2,3),(3,2)] 
NoSwitches = []

AdjacencyList = {1: [2,3], 2: [1,3], 3:[1,2]}

    # Parameters
P = {1 : 2.2, 2 : 10, 3 : 10} # P[1]은 사용되지 않음
Q = {1: 3, 2 : 10, 3 : 10} # Q[1]은 사용되지 않음
G = {(1,2) : 0, (2,1) : 0, (2,3) : 0, (3,2) : 0, 1: 1, 2: 1, 3: 1} 
B = {(1,2) : 10, (2,1) : 10, (2,3) : 10, (3,2) : 10, 1: 1, 2: 1, 3: 1} 
R = {(1,2) : 0.001, (2,1) : 0.001, (2,3) : 0.001, (3,2) : 0.001, (1,3) : 0.001, (3,1) :0.001}
X = {(1,2) : 0.001, (2,1) : 0.001, (2,3) : 0.001, (3,2) : 0.001, (1,3) : 0.001, (3,1) :0.001}

n = len(Buses)

In [60]:
model = gp.Model("MIQP_DNR") 

p,q,l,a,b = dict(), dict(), dict(), dict(), dict()
v = dict()
for line in Switches:
    p[line] = model.addVar(name = 'p'+str(line))
    q[line] = model.addVar(name = 'q'+str(line))
    l[line] = model.addVar(name = 'l'+str(line))
    a[line] = model.addVar(name = 'a'+str(line))
    b[line] = model.addVar(vtype=GRB.BINARY, name = 'b'+str(line)) # eq 16

for bus in Buses:
    v[bus] = model.addVar(name = 'v'+str(bus))

for bus in Buses: # eq10
    model.addConstr(
        sum(
            p[(bus, adjbus)] for adjbus in AdjacencyList[bus]
            ) - 
        sum(
            p[(adjbus, bus)] - R[(adjbus, bus)]*l[(adjbus, bus)] for adjbus in AdjacencyList[bus]
            )
        + G[bus]*v[bus]
        == P[bus])

#for bus in Buses: # eq11
#    model.addConstr(
#        sum(
#            q[(bus, adjbus)] for adjbus in AdjacencyList[bus]
#            ) - 
#        sum(
#            q[(adjbus, bus)] - X[(adjbus, bus)]*l[(adjbus, bus)] for adjbus in AdjacencyList[bus]
#            )
#        + B[bus]*v[bus]
#        == -Q[bus])

M = 10000
for line in Switches: # eq12
    bus1, bus2 = line[0], line[1]
    model.addConstr(
        M*(1-a[line]) + v[bus1] -  2*(R[line]*p[line] + X[line]*q[line]) + (R[line]**2 + X[line]**2)*l[line]
        >= v[bus2]
    )
    
for line in Switches: # eq13
    bus1, bus2 = line[0], line[1]
    model.addConstr(
        -M*(1-a[line]) + v[bus1] -  2*(R[line]*p[line] + X[line]*q[line]) + (R[line]**2 + X[line]**2)*l[line]
        <= v[bus2]
    )
     
model.addConstr( # eq 14
    sum(
        a[line] for line in Switches
    ) == n - 1
)

for line in Switches: # eq 15
    model.addConstr(
        b[line] + b[tuple(reversed(line))] == a[line]
    )
    
for bus in NonsubstationBuses: # eq 16 #이거 없어져야 풀리는데 왜그럴까
    model.addConstr(
        sum(
            b[(bus, adjbus)] for adjbus in AdjacencyList[bus]
        ) == 1
    )
    
for bus in SubstationBuses: # eq 17
    for adjbus in AdjacencyList[bus]:
        model.addConstr(
            b[bus, adjbus] == 0
        )
        
for line in Switches: # eq 18
    model.addConstr(
        (p[line]**2 + q[line]**2)
        <= l[line]*v[line[0]]
    )

for line in Switches: # eq 19
    model.addConstr(
        a[line]*1000
        >= l[line]
    )
    
for bus in SubstationBuses: # eq 20
    model.addConstr(
        v[bus] == 1.03
    )

for bus in SubstationBuses: # eq 21
    model.addConstr(
        v[bus] <= 2
    )
    
for bus in NonsubstationBuses: # eq 21
    model.addConstr(
        v[bus] >= 0.4
    )

model.setObjective(sum(R[line]*l[line] for line in Switches), GRB.MINIMIZE) # eq9
model.update()
model.optimize()

KeyError: (0, 1)

In [66]:
Switches

[]

In [77]:
model = gp.Model("MIQP_DNR") 

p,q,l,a = dict(), dict(), dict(), dict()
v = dict()
for line in Switches:
    p[line] = model.addVar(name = 'p'+str(line))
    q[line] = model.addVar(name = 'q'+str(line))
    l[line] = model.addVar(name = 'l'+str(line))
    a[line] = model.addVar(vtype=GRB.BINARY, name = 'a'+str(line))

for bus in Buses:
    v[bus] = model.addVar(name = 'v'+str(bus))

for bus in NonsubstationBuses: # eq10
    model.addConstr(
        sum(
            p[(bus, adjbus)] for adjbus in AdjacencyList[bus]
            ) - 
        sum(
            p[(adjbus, bus)] - R[(adjbus, bus)]*l[(adjbus, bus)] for adjbus in AdjacencyList[bus]
            )
        + G[bus]*v[bus]
        == -P[bus])

for bus in NonsubstationBuses: # eq11
    model.addConstr(
        sum(
            q[(bus, adjbus)] for adjbus in AdjacencyList[bus]
            ) - 
        sum(
            q[(adjbus, bus)] - X[(adjbus, bus)]*l[(adjbus, bus)] for adjbus in AdjacencyList[bus]
            )
        + B[bus]*v[bus]
        == -Q[bus])

M = 100000
for line in Switches: # eq12
    bus1, bus2 = line[0], line[1]
    model.addConstr(
        M*(1-a[line]) + v[bus1] -  2*(R[line]*p[line] + X[line]*q[line]) + (R[line]**2 + X[line]**2)*l[line]
        >= v[bus2]
    )
    
for line in Switches: # eq13
    bus1, bus2 = line[0], line[1]
    model.addConstr(
        -M*(1-a[line]) + v[bus1] -  2*(R[line]*p[line] + X[line]*q[line]) + (R[line]**2 + X[line]**2)*l[line]
        <= v[bus2]
    )
     
model.addConstr( # eq 14
    sum(
        a[line] for line in Switches
    ) == n - len(SubstationBuses)
)


for line in Switches: # eq 18
    model.addConstr(
        (p[line]**2 + q[line]**2)
        <= l[line]*v[line[0]]
    )

for line in Switches: # eq 19
    model.addConstr(
        a[line]*M
        >= l[line]
    )
    
for bus in SubstationBuses: # eq 20
    model.addConstr(
        v[bus] == 1.03
    )

for bus in SubstationBuses: # eq 21
    model.addConstr(
        v[bus] <= 2
    )
    
for bus in NonsubstationBuses: # eq 21
    model.addConstr(
        v[bus] >= 0.4
    )

model.setObjective(sum(R[line]*l[line] for line in Switches), GRB.MINIMIZE) # eq9
model.update()
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads



GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

In [56]:
model.getVars()

[<gurobi.Var p(1, 2) (value 11.230467748700635)>,
 <gurobi.Var q(1, 2) (value 11.230467748701397)>,
 <gurobi.Var l(1, 2) (value 244.89981862480033)>,
 <gurobi.Var a(1, 2) (value 1.0)>,
 <gurobi.Var p(1, 3) (value 11.230467748700635)>,
 <gurobi.Var q(1, 3) (value 11.230467748701397)>,
 <gurobi.Var l(1, 3) (value 244.89981862480033)>,
 <gurobi.Var a(1, 3) (value 1.0)>,
 <gurobi.Var p(2, 1) (value 0.0)>,
 <gurobi.Var q(2, 1) (value 0.0)>,
 <gurobi.Var l(2, 1) (value 0.0)>,
 <gurobi.Var a(2, 1) (value 0.0)>,
 <gurobi.Var p(3, 1) (value 0.0)>,
 <gurobi.Var q(3, 1) (value 0.0)>,
 <gurobi.Var l(3, 1) (value 0.0)>,
 <gurobi.Var a(3, 1) (value 0.0)>,
 <gurobi.Var p(2, 3) (value 0.0)>,
 <gurobi.Var q(2, 3) (value 0.0)>,
 <gurobi.Var l(2, 3) (value 0.0)>,
 <gurobi.Var a(2, 3) (value 0.0)>,
 <gurobi.Var p(3, 2) (value 0.0)>,
 <gurobi.Var q(3, 2) (value 0.0)>,
 <gurobi.Var l(3, 2) (value 0.0)>,
 <gurobi.Var a(3, 2) (value 0.0)>,
 <gurobi.Var v1 (value 1.03)>,
 <gurobi.Var v2 (value 0.98556792864567

In [25]:
if model.status == GRB.INFEASIBLE:
    model.computeIIS()
    model.write("model.ilp")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads


Computing Irreducible Inconsistent Subsystem (IIS)...

           Constraints          |            Bounds           |  Runtime
      Min       Max     Guess   |   Min       Max     Guess   |
--------------------------------------------------------------------------
        0        42         -         0        27         -           0s
        9         9         9         0         0         0           0s

IIS computed: 9 constraints, 0 bounds
IIS runtime: 0.00 seconds (0.00 work units)
